In [342]:
import pandas as pd
from AUIDFunctions import *
import inspect
uh = pd.read_csv("input/unit-history_demo.csv")
am = pd.read_csv("input/all-members_demo.csv")
ac = pd.read_csv("input/accused_demo.csv", dtype={"Middle.Initial":str})
am = am.rename(columns = {'TID' : 'ID1'})
uh = uh.rename(columns = {'TID' : 'ID2'})

FileNotFoundError: File b'input/unit-history_demo.csv' does not exist

In [343]:
pd.set_option('display.max_columns', 30)

In [131]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [132]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df



In [133]:
amd = am.dropna(axis=1, how='all')
uhd = uh.dropna(axis=1, how='all')
amdm = AddColumns(amd)
uhdm = AddColumns(uhd)
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
print(cols)

Index(['ID1', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'F4FN', 'F4LN',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
Index(['ID2', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Unit', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1']


In [134]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 29)
(4289, 29)


In [295]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2'], return_unmatched = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmatched:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [136]:
m = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
])

# common names/nicknames abbreviations/translations

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4289
******
******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
1312
******
******
['F4FN', 'F4LN', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
19949
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
6011
******
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
3
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Middle.Initial', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
******
['First.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Dat

In [137]:
uid_ref = pd.concat([m[0][["ID1", "ID2"]], m[1][["ID1"]], m[2][["ID2"]]]).reset_index(drop=True)
if "UID" not in uid_ref.columns:
    uid_ref.insert(0, "UID", uid_ref.index + 1)
am.head()

,ID1,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Gender,Race,Birth.Year
0,1,JEFFERY,AARON,M,NaN,2005-09-26,MALE,WHITE,1971
1,2,KARINA,AARON,NaN,NaN,2005-09-26,FEMALE,HISPANIC,1980
2,3,DANIEL,ABATE,P,NaN,1970-06-15,MALE,WHITE,1942
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,MALE,WHITE,1968
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,MALE,WHITE,1942


In [138]:
uid_ref.head()

,UID,ID1,ID2
0,1,2,2
1,2,16,16
2,3,17,17
3,4,32,31
4,5,33,32


In [139]:
am = am.merge(uid_ref[["ID1", "UID"]], on='ID1', how='left')
uh = uh.merge(uid_ref[["ID2", "UID"]], on='ID2', how='left')

In [140]:
ref = pd.concat([am, uh])
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [141]:
ref['Middle.Initial'] = ref['Middle.Initial'].fillna("")
print(ref.columns)
ref.to_csv("test-reference.csv", index=False)

Index(['Appointed.Date', 'Birth.Year', 'Current.Age', 'Current.Unit',
       'First.Name', 'Gender', 'ID1', 'ID2', 'Last.Name', 'Middle.Initial',
       'Race', 'Star1', 'Star10', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6',
       'Star7', 'Star8', 'Star9', 'Suffix.Name', 'UID'],
      dtype='object')


In [142]:
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [144]:
'''profiles = AggregateData(ref, uid = "UID", id_cols = [],
              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'''

'profiles = AggregateData(ref, uid = "UID", id_cols = [],\n              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'

In [145]:
uid = "UID"
uid_col = [uid]
agg_df = ref[uid_col].drop_duplicates().reset_index(drop=True) 
max_cols = setdiff(ref.columns, ["UID"])
print(max_cols)

['Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2', 'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit', 'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender', 'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10']


In [146]:
profiles = ref[uid_col + max_cols].drop_duplicates()
profiles = profiles.groupby(uid_col, as_index=False)[max_cols]

In [147]:
profiles = profiles.agg(max)

In [148]:
profiles.columns

Index(['UID', 'Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2',
       'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit',
       'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender',
       'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10'],
      dtype='object')

In [149]:
acd = ac.dropna(axis=1, how='all')
acd = AddColumns(acd)
profiles = AddColumns(profiles)

In [191]:
#def StarMerge(multistar,singlestar, max_star, on_list, keep_columns = ["ID1", "ID2"]):
acdt = acd.loc[acd["Star"]>0]
pt = profiles.loc[(profiles["Star1"].notnull()) & (profiles["Star1"] > 0)]

In [201]:
mc = ['First.Name', 'Last.Name','Appointed.Date', 'Gender', "Race"]
mt = acdt.merge(pt, how = "inner", left_on = mc + ["Star"], right_on= mc + ["Star1"])

In [336]:
keep_columns = ["TID", "UID"]
on_list =  [['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Birth.Year"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender',"Birth.Year"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender',"Birth.Year"],
    ['F4FN', 'F4LN', "Middle.Initial", 'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race", "Current.Unit","Birth.Year"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race", "Current.Unit","Birth.Year"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender',"Birth.Year"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name', 'Last.Name','Appointed.Date', 'Gender', "Birth.Year"],
    ['First.Name', 'Last.Name','Appointed.Date', "Birth.Year"],
    ['First.Name','Appointed.Date', 'Gender', "Race","Birth.Year"],
    ['First.Name','Appointed.Date', 'Gender',"Birth.Year"]]

In [338]:
ac = pd.read_csv("input/accused_demo.csv", dtype={"Middle.Initial":str})
ac = AddColumns(ac)

In [339]:
def StarMerge(df1, df2, on_list, keep_columns, return_unmerged = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    df2 = df2.loc[(df2["Star"].notnull()) & (df2["Star"] > 0)]
    stars = ["Star" + str(i) for i in range(1,11)]
    for mc_cols in on_list:
        for star in stars:
            mc1 = mc_cols + [star]
            mc2 = mc_cols + ["Star"]
            df1t = RemoveDuplicates(df1.loc[(df1[star].notnull()) & (df1[star] > 0), keep_columns[:1] + mc1], mc1)
            df2t = RemoveDuplicates(df2[keep_columns[1:] + mc2], mc2)
            dfmt = df1t.merge(df2t, left_on = mc1, right_on = mc2, how='inner')
            if dfmt.shape[0] > 0:
                print('******')
                print(mc1)
                print(dfmt.shape[0])
                print('******')
                dfmt['Match'] = '-'.join(mc + [star + "/" + "Star"])
                dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
                df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
                df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmerged:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [340]:
m = StarMerge(profiles, ac, on_list, keep_columns = ["UID", "TID"], return_unmerged=False)

m2 = LoopMerge(profiles.loc[~profiles["UID"].isin(m["UID"])], ac.loc[~ac["TID"].isin(m["TID"])], on_list, keep_columns = ["UID", "TID"])

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star1']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star1']
6084
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star2']
1301
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star3']
476
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star4']
155
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star5']
48
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star6']
12
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'Star7']
2
******
******
['First.Name', 'Las

In [341]:
print(m.shape)
print(m2[0].shape)

(10025, 3)
(5987, 3)


In [334]:
print(m2[2])

         TID First.Name    Last.Name Middle.Initial Suffix.Name  \
1453    1454     VALENA      BRADLEY            NaN         NaN   
4275    4276       JOSE     CHAMBERS            NaN         NaN   
4462    4463   MICHONNE        BERRY            NaN         NaN   
7768    7769     SUSANA  LACASACALIZ            NaN         NaN   
9246    9247       LUIS    CONTRERAS            NaN         NaN   
12072  12073   CAROLINE     DEWINTER            NaN         NaN   
14570  14571      JAMES      BANSLEY            NaN         NaN   
14713  14714      JAMES      BANSLEY            NaN         NaN   
15031  15032      JAMEL        JUDEH            NaN         NaN   
15180  15181       ERIC       VALDES            NaN         NaN   

      Appointed.Date  Birth.Year  Gender      Race  Current.Unit Current.Rank  \
1453      1989-12-26        1964  FEMALE   UNKNOWN             2           PO   
4275      2000-01-24        1972    MALE     BLACK          -999           PO   
4462      1999-11-2

In [330]:
print(m2[1].loc[m2[1]["First.Name"]=="JOSE"].sort_values("Last.Name"))

         UID Appointed.Date      ID2  Star7   Star4  Star9    Star2  Star5  \
5889    5890     2006-05-01    481.0 -999.0  -999.0 -999.0   -999.0 -999.0   
6368    6369     2013-08-26   1251.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8830    8831     2015-10-26   5068.0 -999.0  -999.0 -999.0   -999.0 -999.0   
9826    9827     2015-10-26   6589.0 -999.0  -999.0 -999.0   -999.0 -999.0   
831      832     1991-02-27   6699.0 -999.0  4888.0 -999.0  21241.0 -999.0   
1164    1165     2015-04-06   9398.0 -999.0  -999.0 -999.0   -999.0 -999.0   
31693  31694     2016-02-29      NaN    NaN     NaN    NaN      NaN    NaN   
31695  31696     2016-06-27      NaN    NaN     NaN    NaN      NaN    NaN   
31710  31711     2016-02-29      NaN    NaN     NaN    NaN      NaN    NaN   
1401    1402     2015-02-02  10969.0 -999.0  -999.0 -999.0   -999.0 -999.0   
12579  12580     2013-12-02  10968.0 -999.0  -999.0 -999.0  14405.0 -999.0   
4759    4760     2007-07-30  11005.0 -999.0  -999.0 -999.0   -99

In [329]:
print(profiles.loc[(profiles["First.Name"].str.contains("JOSE")) & (profiles["Last.Name"].str.contains("CH"))].sort_values("Last.Name"))

         UID Appointed.Date      ID2  Star7   Star4  Star9    Star2  Star5  \
4348    4349     1953-03-02   1000.0 -999.0  -999.0 -999.0   -999.0 -999.0   
26023  26024     1961-06-05   2615.0 -999.0  -999.0 -999.0   -999.0 -999.0   
421      422     1954-02-11   3424.0 -999.0  -999.0 -999.0   -999.0 -999.0   
7953    7954     2000-02-28   3671.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8357    8358     2000-01-24   4324.0 -999.0  -999.0 -999.0   -999.0 -999.0   
26392  26393     1953-03-02   4325.0 -999.0  -999.0 -999.0   -999.0 -999.0   
26397  26398     1990-03-26   4348.0 -999.0  -999.0 -999.0   -999.0 -999.0   
26405  26406     1957-07-16   4395.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8399    8400     2001-02-05   4394.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8409    8410     1966-11-14   4416.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8424    8425     1968-08-26   4438.0 -999.0  -999.0 -999.0   -999.0 -999.0   
8436    8437     1973-06-04   4456.0 -999.0  -999.0 -999.0   103